In [204]:
import (
	"math"
    "time"
    "fmt"

	"golang.org/x/exp/rand"

	"gonum.org/v1/gonum/stat/distuv"
)

const ThiefCapacity = 3_000


func Max(a, b int64) int64 {
	if a > b {
		return a
	}
	return b
}

func Min(a, b int64) int64 {
	if a < b {
		return a
	}
	return b
}

func MinInt32(a, b int32) int32 {
	if a < b {
		return a
	}
	return b
}

func MaxInt32(a, b int32) int32 {
	if a > b {
		return a
	}
	return b
}


type Heist struct {
	Guards      int32
	Thieves     int32
	TargetCoins int32

	ThiefEvadeBonus      float64
	ThiefCapacityBonus   float64
	GuardEfficiencyBonus float64
	GuardAwarenessBonus float64
}

type HeistOutcome struct {
	Loot              int64
	SuccessfulThieves int32
	CaughtThieves     int32
	SleepingGuards    int32
}

func CalculateHeist(h Heist, rsrc rand.Source) HeistOutcome {
	guards := h.Guards
	thieves := h.Thieves

	guardsf := float64(guards)
	dist := distuv.Binomial{
		N:   guardsf,
		P:   0.075 * (1 - h.GuardAwarenessBonus),
		Src: rsrc,
	}
	sleepingGuards := MinInt32(MinInt32(int32(dist.Rand()), (guards+1)/3), thieves)

	guards = guards - sleepingGuards
	guardsf = float64(guards)
	thievesf := float64(thieves)

	guardsp := guardsf * (h.GuardEfficiencyBonus + 1) / 3
	thievesp := thievesf * (h.ThiefEvadeBonus + 1)

	dist = distuv.Binomial{
		N:   thievesf,
		P:   thievesp / (thievesp + guardsp),
		Src: rsrc,
	}
	successfulThieves := int32(dist.Rand())
	caughtThieves := MinInt32(thieves - successfulThieves, guards)
	guardsProtectingLoot := float64(MaxInt32(guards-caughtThieves, 0))
	thiefEfficiency := 0.5 + 0.5/(1+math.Pow(guardsProtectingLoot/12, 0.7))

	maxLoot := int32(float64(successfulThieves) * ThiefCapacity * (h.ThiefCapacityBonus + 1) * thiefEfficiency)
	loot := int64(MinInt32(maxLoot, h.TargetCoins))

	return HeistOutcome{
		Loot:              loot,
		SuccessfulThieves: successfulThieves,
		CaughtThieves:     caughtThieves,
		SleepingGuards:    sleepingGuards,
	}
}


In [262]:
rng := rand.NewSource(uint64(time.Now().UnixNano()))

r := CalculateHeist(Heist{
    Thieves: 70,
    Guards: 20,
    TargetCoins: 1_000_000,
}, rng)

fmt.Printf("%+v\n", r)

fmt.Printf("Profit = %d\n", int32(r.Loot) - r.CaughtThieves * 12000)
fmt.Printf("Overall = %d\n", int32(r.Loot) - r.CaughtThieves * 12000 + r.SleepingGuards * 8000)


{Loot:145831 SuccessfulThieves:66 CaughtThieves:4 SleepingGuards:2}
Profit = 97831
Overall = 113831


17 <nil>